In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt
import time

In [2]:
N_wn = 20
lbound_wn = 0
ubound_wn = 10
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [3]:
center_ = 2.0

In [4]:
h0 = 0.03

results = []

sphere1 = bempp.api.shapes.sphere(h=h0, origin=(center_, 0, 0))
sphere2 = bempp.api.shapes.sphere(h=h0, origin=(-center_, 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

In [5]:
start = time.time()
for index2, point2 in enumerate(quad_points):

    wavenumber = -1j * np.log(point2)

    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat2 = np.block([[mat11,mat12_zero],[mat21_zero,mat22]])

    combined = np.linalg.inv(mat2).dot(mat)
    sign, logdet = slogdet(combined)
    
    results.append(logdet)
    print(index2, logdet)
end = time.time()
print(end - start)

0 4.5230486023228857e-13
1 -2.9868473141785907e-07
2 -4.976170613281056e-06
3 -2.6041522625070455e-05
4 -8.481050700328962e-05
5 -0.0002130367561225327
6 -0.00045421925923874826
7 -0.0008651892419814217
8 -0.0015180978015221687
9 -0.0025029657649958386
10 -0.003931019012311952
11 -0.005939123746771846
12 -0.008695770864929708
13 -0.012409259495837814
14 -0.01733903431739294
15 -0.023811601565111917
16 -0.03224319148179074
17 -0.04317253916166107
18 -0.057309168988975485
19 -0.07560605851843825
9044.07748246193


In [6]:
print(np.trapz(-np.array(results)/quad_points, quad_points)/(2*np.pi))

0.0025318251812500112


In [ ]:
N = 20 h = 0.03
integral from 0 to 10
center = 1.2 time is 11933.562218666077 and CasE is 0.07448533782808972
center = 1.4 time is 7982.937509298325s   and CasE is 0.017155131855374216
center = 1.6 time is  8041.89542889595s and CasE is 0.0074806214626398585
center = 1.8 time is  7991.877442359924  and CasE is 0.004088635881154157
center = 2.0 time is   9044.07748246193 and CasE is 0.0025318251812500112